In [1]:
import copy
import pathlib
from pdb import set_trace

import numpy as np
import pybullet as p
import pybullet_data
from pinocchio.robot_wrapper import RobotWrapper

from controllers.feedback_control import FeedbackController
from controllers.utils import get_state_update_pinocchio, send_joint_command

In [2]:
p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -9.81)

# Load plane
p.loadURDF("plane.urdf")

# Load Franka Panda Robot
robotID = p.loadURDF("./franka_panda/panda.urdf", useFixedBase=True)

# Get number of joints
n_j = p.getNumJoints(robotID)

debug_sliders = []
joint_ids = []

for i in range(n_j):
    # get info of each joint
    _joint_infos = p.getJointInfo(robotID, i)

    if _joint_infos[2] != p.JOINT_FIXED:
        # Add a debug slider for all non-fixed joints
        debug_sliders.append(
            p.addUserDebugParameter(
                _joint_infos[1].decode("UTF-8"),  # Joint Name
                _joint_infos[8],  # Lower Joint Limit
                _joint_infos[9],  # Upper Joint Limit
                0.0,  # Default Joint Angle
            )
        )

        # Save the non-fixed joint IDs
        joint_ids.append(_joint_infos[0])

In [3]:
while True:
    for slider_id, joint_id in zip(debug_sliders, joint_ids):
        # Get joint angle from debug slider
        try:
            _joint_angle = p.readUserDebugParameter(slider_id)
        except:
            # Sometimes it fails to read the debug slider
            continue

        # Apply joint angle to robot
        p.resetJointState(robotID, joint_id, _joint_angle)

    p.stepSimulation()

KeyboardInterrupt: 